In [1]:
!pip install selenium
!pip install webdriver_manager
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv  # Import csv module

# Set up ChromeDriver
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Open the target URL
url = 'https://www.consumeraffairs.com/travel/marriott.html'
driver.get(url)

# Counter for review numbers
review_counter = 1

# Create a CSV file to save the review data
with open('marriott_reviews.csv', mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Review No', 'Customer Name', 'Date', 'Stars', 'Review', 'Services Rated', 'Verified Purchase']  # Add new fields
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()

    # Function to scrape reviews from a page
    def scrape_reviews():
        global review_counter  # Access the counter variable

        # Wait for reviews to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'rvw__cntr'))
        )

        # Click all "More" buttons to reveal full reviews (if any)
        more_buttons = driver.find_elements(By.CLASS_NAME, 'js-expander-container')
        for button in more_buttons:
            try:
                button.click()
                time.sleep(1)  # Wait for the review to expand
            except:
                pass  # Ignore if the button is not clickable

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = soup.find_all('div', class_='rvw__cntr')

        # Extract and write the review data to the CSV
        for review in reviews:
            # Extract customer name
            customer_name_tag = review.find('span', class_='rvw__inf-nm')
            customer_name = customer_name_tag.text.strip() if customer_name_tag else "Anonymous"

            # Extract date
            date = review.find('p', class_='rvw__rvd-dt')
            review_date = date.text.strip() if date else "No date"

            # Extract star rating from class name
            stars_div = review.find('div', class_='rvw__rtg')
            star_rating = "No rating"
            if stars_div:
                star_classes = stars_div.get('class', [])
                for cls in star_classes:
                    if 'stars-sprt--p-' in cls:
                        star_rating = cls.split('--p-')[-1] + " stars"

            # Extract and clean the review text
            review_text = review.find('div', class_='rvw__top-text')
            review_content = review_text.text.strip() if review_text else "No review content"
            # Remove 'More' if it appears at the end
            if review_content.endswith('More'):
                review_content = review_content[:-4].strip()

            # Extract service tags (e.g., Price, Punctuality & Speed, Staff)
            services_rated = []
            service_tags = review.find_all('span', class_='rvw__tag')
            for tag in service_tags:
                services_rated.append(tag.text.strip())
            services_rated_str = ', '.join(services_rated) if services_rated else "No services rated"

            # Check if the review is marked as "Verified purchase"
            verified_purchase = review.find('span', class_='rvw__inf-ver')
            verified = 'Yes' if verified_purchase and 'Verified purchase' in verified_purchase.text.strip() else 'No'

            # Print the extracted data
            print(f"Review No {review_counter}:")
            print(f"Customer: {customer_name}")
            print(f"Date: {review_date}")
            print(f"Stars: {star_rating}")
            print(f"Review: {review_content}")
            print(f"Services Rated: {services_rated_str}")
            print(f"Verified Purchase: {verified}")
            print("-" * 80)

            # Write the review data to the CSV file
            writer.writerow({
                'Review No': review_counter,
                'Customer Name': customer_name,
                'Date': review_date,
                'Stars': star_rating,
                'Review': review_content,
                'Services Rated': services_rated_str,
                'Verified Purchase': verified
            })

            # Increment the review counter
            review_counter += 1

    # Loop through pages
    for page_num in range(1, 6):  # Assuming there are 5 pages
        if page_num > 1:
            try:
                # Locate the pagination link for the next page
                next_page_link = driver.find_element(By.LINK_TEXT, str(page_num))
                
                # Scroll the element into view and click it using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_link)
                time.sleep(1)  # Allow time for scroll
                driver.execute_script("arguments[0].click();", next_page_link)

                time.sleep(2)  # Wait for the next page to load
            except Exception as e:
                print(f"Error on page {page_num}: {e}")
                continue
        
        scrape_reviews()

driver.quit()

Review No 1:
Customer: Larra
Date: Reviewed Oct. 20, 2024
Stars: 4 stars
Review: I have had Marriott Bonvoy card with points for about 3 years now. I have never had any trouble booking or canceling a reservation. If you use their app the booking results in cheaper points for the room. You have 24 hours prior to cancel with no fee and they return the points to your card. We have used them for vacation and my husband drives a truck so on the fly it is super easy if he needs a room for the night to see what is available and use our points. We haven't had an issue with Marriott hotels. We never book through a 3rd party.
Services Rated: Online & App
Verified Purchase: No
--------------------------------------------------------------------------------
Review No 2:
Customer: Brigina
Date: Reviewed May 21, 2024
Stars: 4 stars
Review: It's sad to say that the customer service they are provided by the gentleman David at the front counter was excellent, but they should reconsider their policy of 

In [3]:
!pip install faker

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import csv
import random
from faker import Faker

fake = Faker()

dummy_reviews_count = 500  

existing_reviews = []
with open('marriott_reviews.csv', mode='r', encoding='utf-8') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        existing_reviews.append(row)

# Start counter from the last existing review number
if existing_reviews:
    start_review_no = len(existing_reviews) + 1
else:
    start_review_no = 1

# Define review templates for different star ratings
review_templates = {
    5: [
        "Absolutely wonderful experience! {name} was amazing.",
        "Everything was perfect. Highly recommend!",
        "I loved my stay! The service was exceptional.",
        "This is the best hotel I've ever stayed at. {name} was fantastic!"
    ],
    4: [
        "I had a great experience overall, just a few minor issues.",
        "Very nice hotel, but could improve on the service.",
        "Great stay! I would come back again.",
        "The staff were friendly, and the room was clean."
    ],
    3: [
        "It was okay, but nothing special. Could be better.",
        "Average experience, but I might consider staying again.",
        "The location was good, but the service was lacking.",
        "It was a decent stay, but I expected more."
    ],
    2: [
        "I was disappointed with my stay. The service was poor.",
        "Not worth the price. I expected better quality.",
        "Service was lacking, and I had issues with my room.",
        "I wouldn’t recommend this hotel."
    ],
    1: [
        "Horrible experience. I will not return.",
        "Very dissatisfied. The room was not clean.",
        "Awful service and unfriendly staff.",
        "I regret staying here. Totally unacceptable."
    ]
}

with open('marriott_reviews.csv', mode='a', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Review No', 'Customer Name', 'Date', 'Stars', 'Review', 'Services Rated', 'Verified Purchase']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    for i in range(dummy_reviews_count):
        review_no = start_review_no + i
        customer_name = fake.name()
        review_date = fake.date_between(start_date='-1y', end_date='today')  
        star_rating = random.randint(1, 5)
        review_template = random.choice(review_templates[star_rating])
        review_content = review_template.format(name=customer_name)
        services_rated = random.sample(['Price', 'Punctuality & Speed', 'Staff', 'Cleanliness', 'Amenities'], k=random.randint(1, 3))
        services_rated_str = ', '.join(services_rated)
        verified = random.choice(['Yes', 'No'])
        writer.writerow({
            'Review No': review_no,
            'Customer Name': customer_name,
            'Date': review_date,
            'Stars': f"{star_rating} stars",
            'Review': review_content,
            'Services Rated': services_rated_str,
            'Verified Purchase': verified
        })

print(f"Added {dummy_reviews_count} dummy reviews to the CSV file.")

Added 500 dummy reviews to the CSV file.


In [5]:
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install gensim
!pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from gensim.models import LdaModel
from sklearn.decomposition import NMF
from gensim.corpora import Dictionary
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Download NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
import pandas as pd
df=pd.read_csv('marriott_reviews.csv')
df

,Review No,Customer Name,Date,Stars,Review,Services Rated,Verified Purchase
0,1,Larra,"Reviewed Oct. 20, 2024",4 stars,I have had Marriott Bonvoy card with points fo...,Online & App,No
1,2,Brigina,"Reviewed May 21, 2024",4 stars,It's sad to say that the customer service they...,"Customer Service, Price",Yes
2,3,Karena,"Reviewed April 9, 2024",5 stars,"My husband, nephew, and I stayed at the Fairfi...",Staff,No
3,4,Deanna,"Reviewed Jan. 11, 2024",5 stars,Eddie (Eduardo) at the Miami Dolphin Mall Marr...,Staff,No
4,5,Ruby,"Reviewed Nov. 22, 2023",5 stars,1st of all I will like to thank Marriott Hotel...,Staff,No
...,...,...,...,...,...,...,...
545,546,Jamie Coleman,2024-05-29,1 stars,Very dissatisfied. The room was not clean.,"Punctuality & Speed, Staff",No
546,547,Chelsey Sanchez,2024-02-21,2 stars,I was disappointed with my stay. The service w...,"Punctuality & Speed, Cleanliness",Yes
547,548,Hannah Morris,2024-07-20,2 stars,Not worth the price. I expected better quality.,"Punctuality & Speed, Staff",Yes
548,549,Marc Riggs,2024-01-07,4 stars,Great stay! I would come back again.,Cleanliness,Yes


In [8]:
df.isnull().sum()

Review No            0
Customer Name        0
Date                 0
Stars                0
Review               0
Services Rated       0
Verified Purchase    0
dtype: int64

In [9]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

df['Cleaned_Review'] = df['Review'].apply(clean_text)

# Remove stop words and perform lemmatization
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['Processed_Review'] = df['Cleaned_Review'].apply(preprocess_text)

In [10]:
# Topic Modeling using LDA
# Tokenize reviews for LDA
tokenized_reviews = [review.split() for review in df['Processed_Review']]

# Create a dictionary and corpus for LDA
dictionary = Dictionary(tokenized_reviews)
corpus = [dictionary.doc2bow(review) for review in tokenized_reviews]

In [11]:
# Train the LDA model
num_topics = 5  # You can adjust this number based on your needs
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=10)

# Print topics
for idx, topic in lda_model.print_topics(num_words=5):
    print(f"Topic #{idx + 1}: {topic}")

Topic #1: 0.051*"stay" + 0.032*"service" + 0.026*"room" + 0.025*"back" + 0.023*"come"
Topic #2: 0.046*"hotel" + 0.031*"wouldnt" + 0.030*"recommend" + 0.030*"good" + 0.028*"service"
Topic #3: 0.048*"experience" + 0.032*"hotel" + 0.029*"issue" + 0.025*"great" + 0.025*"overall"
Topic #4: 0.083*"service" + 0.050*"hotel" + 0.037*"stay" + 0.034*"poor" + 0.034*"disappointed"
Topic #5: 0.043*"room" + 0.042*"clean" + 0.032*"wonderful" + 0.032*"absolutely" + 0.032*"amazing"


In [12]:
# Using NMF for Topic Modeling
# Create TF-IDF features
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(df['Processed_Review'])

# Train the NMF model
num_topics_nmf = 4  # Adjust this number based on your needs
nmf_model = NMF(n_components=num_topics_nmf, random_state=42)
W = nmf_model.fit_transform(X_tfidf)  # Document-topic matrix
H = nmf_model.components_  # Topic-word matrix

# Print NMF topics
print("\nNMF Topics:")
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(H):
    print(f"Topic #{topic_idx + 1}: " + " ".join([feature_names[i] for i in topic.argsort()[-5:]]))



NMF Topics:
Topic #1: lacking poor disappointed stay service
Topic #2: great overall minor issue experience
Topic #3: stay great would back come
Topic #4: worth price quality better expected


In [13]:
# Multi-Label Classification Preparation
# Create labels based on keywords in the review text (e.g., 'product', 'service', 'payment', 'technical')
def classify_labels(review):
    labels = []
    if any(word in review for word in ['product', 'price', 'item']):
        labels.append('product-related')
    if any(word in review for word in ['service', 'staff', 'customer']):
        labels.append('service-related')
    if any(word in review for word in ['payment', 'bill', 'charge']):
        labels.append('payment-related')
    if any(word in review for word in ['app', 'website', 'technical']):
        labels.append('technical-issue')
    return labels

df['Labels'] = df['Processed_Review'].apply(classify_labels)
print(df['Labels'])

0                       [technical-issue]
1      [service-related, payment-related]
2                       [service-related]
3                       [technical-issue]
4                       [service-related]
                      ...                
545                                    []
546    [service-related, technical-issue]
547                     [product-related]
548                                    []
549                                    []
Name: Labels, Length: 550, dtype: object


In [14]:
# Binarize the labels for multi-label classification
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Labels'])

In [15]:
# Feature Engineering with TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500)
X = tfidf_vectorizer.fit_transform(df['Processed_Review']).toarray()

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, hamming_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# Features and target
X = df['Review']
y = df['Labels']

# Convert y to binary format
mlb = MultiLabelBinarizer()
y_binary = mlb.fit_transform(y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)

# Logistic Regression Pipeline
pipeline_lr = Pipeline([
    ('tfidf', tfidf),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
])
param_grid_lr = {
    'clf__estimator__C': [0.01, 0.1, 1, 10],
}

# Training and evaluation
print("Training Logistic Regression...")
grid_search_lr = GridSearchCV(pipeline_lr, param_grid_lr, scoring='f1_micro', cv=5)
grid_search_lr.fit(X_train, y_train)

# Best parameters and evaluation on test set
print(f"Best Parameters for Logistic Regression: {grid_search_lr.best_params_}")
y_pred_lr = grid_search_lr.predict(X_test)

print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr))


Training Logistic Regression...
Best Parameters for Logistic Regression: {'clf__estimator__C': 10}
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        39
           3       1.00      1.00      1.00        16

   micro avg       1.00      1.00      1.00        68
   macro avg       1.00      1.00      1.00        68
weighted avg       1.00      1.00      1.00        68
 samples avg       0.47      0.47      0.47        68



C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Pipeline
pipeline_rf = Pipeline([
    ('tfidf', tfidf),
    ('clf', OneVsRestClassifier(RandomForestClassifier()))
])
param_grid_rf = {
    'clf__estimator__n_estimators': [100, 200],
    'clf__estimator__max_depth': [None, 10, 20],
}

# Training and evaluation
print("Training Random Forest...")
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, scoring='f1_micro', cv=5)
grid_search_rf.fit(X_train, y_train)

# Best parameters and evaluation on test set
print(f"Best Parameters for Random Forest: {grid_search_rf.best_params_}")
y_pred_rf = grid_search_rf.predict(X_test)

print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))



Training Random Forest...
Best Parameters for Random Forest: {'clf__estimator__max_depth': None, 'clf__estimator__n_estimators': 200}
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        39
           3       1.00      0.94      0.97        16

   micro avg       1.00      0.99      0.99        68
   macro avg       1.00      0.98      0.99        68
weighted avg       1.00      0.99      0.99        68
 samples avg       0.46      0.46      0.46        68



C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
from sklearn.svm import SVC

# Support Vector Classifier Pipeline
pipeline_svc = Pipeline([
    ('tfidf', tfidf),
    ('clf', OneVsRestClassifier(SVC(probability=True)))
])
param_grid_svc = {
    'clf__estimator__C': [0.1, 1, 10],
    'clf__estimator__kernel': ['linear', 'rbf']
}

# Training and evaluation
print("Training Support Vector Classifier...")
grid_search_svc = GridSearchCV(pipeline_svc, param_grid_svc, scoring='f1_micro', cv=5)
grid_search_svc.fit(X_train, y_train)

# Best parameters and evaluation on test set
print(f"Best Parameters for SVC: {grid_search_svc.best_params_}")
y_pred_svc = grid_search_svc.predict(X_test)

print("SVC Classification Report:")
print(classification_report(y_test, y_pred_svc))


Training Support Vector Classifier...
Best Parameters for SVC: {'clf__estimator__C': 10, 'clf__estimator__kernel': 'rbf'}
SVC Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        39
           3       1.00      0.94      0.97        16

   micro avg       1.00      0.99      0.99        68
   macro avg       1.00      0.98      0.99        68
weighted avg       1.00      0.99      0.99        68
 samples avg       0.46      0.46      0.46        68



C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
from sklearn.naive_bayes import MultinomialNB

# Multinomial Naive Bayes Pipeline
pipeline_nb = Pipeline([
    ('tfidf', tfidf),
    ('clf', OneVsRestClassifier(MultinomialNB()))
])
param_grid_nb = {
    'clf__estimator__alpha': [0.5, 1.0]
}

# Training and evaluation
print("Training Multinomial Naive Bayes...")
grid_search_nb = GridSearchCV(pipeline_nb, param_grid_nb, scoring='f1_micro', cv=5)
grid_search_nb.fit(X_train, y_train)

# Best parameters and evaluation on test set
print(f"Best Parameters for Multinomial Naive Bayes: {grid_search_nb.best_params_}")
y_pred_nb = grid_search_nb.predict(X_test)

print("Multinomial Naive Bayes Classification Report:")
print(classification_report(y_test, y_pred_nb))


Training Multinomial Naive Bayes...
Best Parameters for Multinomial Naive Bayes: {'clf__estimator__alpha': 0.5}
Multinomial Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00        10
           2       0.95      1.00      0.97        39
           3       1.00      1.00      1.00        16

   micro avg       0.97      1.00      0.99        68
   macro avg       0.99      1.00      0.99        68
weighted avg       0.97      1.00      0.99        68
 samples avg       0.47      0.47      0.47        68



C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
